In [211]:
import platform
from collections import Counter

import numpy as np

# tokenizer import
from konlpy.tag import Okt, Komoran, Hannanum, Kkma

#운영체제에 따라 mecab설치 방법이 다름.
if platform.system() == "Windows":
    try:
        from eunjeon import Mecab
    except:
        print("please install eunjeon module")
else:  # Ubuntu일 경우
    from konlpy.tag import Mecab

from typing import List, Callable, Union, Any, TypeVar, Tuple, Dict

from functools import partial
from sklearn.feature_extraction.text import CountVectorizer


In [212]:
with open("./sents.txt", "r") as f:
    sents = f.read().split("\n")
print(sents)

['오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다', '서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다', '경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다', '이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다', '성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다', '이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다', '5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다', '용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기', '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다', '김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 

In [213]:
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    elif tokenizer_name == "kkma":
        tokenizer = Kkma()
    else:
        tokenizer = Mecab()
    return tokenizer

In [214]:
def get_tokens(sent: List[str], noun=True, tokenizer="mecab") -> List[str]:
    tokenizer = get_tokenizer(tokenizer)

    if noun:
        nouns = tokenizer.nouns(sent)
        print('n',nouns)
        nouns = [word for word in nouns if len(word) > 1]
        print('nouns',nouns)
        return nouns
    
    return tokenizer.morphs(sent)


In [216]:
#stopwords = ["연합뉴스", "가방"]

#vectorizer = CountVectorizer(
    #stop_words=stopwords,
    #tokenizer=partial(get_tokens, noun=True, tokenizer="mecab"),
    #min_df=2,
#)
# 코퍼스로부터 각 단어의 빈도 수를 기록.
#x = vectorizer.fit_transform(sents)
#print('x',x)
#x.toarray().shape
#print('x',x.toarray())


# 각 단어의 인덱스가 어떻게 부여되었는지를 보여줌.
#vocab_idx = vectorizer.vocabulary_
#print('v',vocab_idx)
#idx_vocab = {idx: vocab for vocab, idx in vocab_idx.items()}
#print(idx_vocab)

In [219]:
def vectorize_sents(
    sents: List[str], stopwords=None, min_count=2, tokenizer="mecab", noun=True
):

    vectorizer = CountVectorizer(
        stop_words=stopwords,
        tokenizer=partial(get_tokens, noun=noun, tokenizer="mecab"),
        min_df=min_count,
    )

    vec = vectorizer.fit_transform(sents)
    print(vec)
    vocab_idx = vectorizer.vocabulary_
    print(vocab_idx)
    idx_vocab = {idx: vocab for vocab, idx in vocab_idx.items()}
    print(idx_vocab)
    
    return vec, vocab_idx, idx_vocab

In [220]:
#stopwords = ["연합뉴스", "가방"]
#x, vocab_idx, idx_vocab = vectorize_sents(sents, stopwords)

#print('x',x)


In [221]:
def word_similarity_matrix(x, min_sim=0.3):
    sim_mat = 1 - pairwise_distances(x.T, metric="cosine")
    sim_mat[np.where(sim_mat <= min_sim)] = 0

    return sim_mat

In [222]:
mat = word_similarity_matrix(x)

In [223]:
def word_graph(
    sents: List[str],
    min_count=2,
    min_sim=0.3,
    tokenizer="mecab",
    noun=True,
    stopwords: List[str] = ["연합뉴스", "그리고", "기자"],
):

    mat, vocab_idx, idx_vocab = vectorize_sents(
        sents, stopwords, min_count=min_count, tokenizer=tokenizer, noun=noun
    )

    mat = word_similarity_matrix(mat, min_sim=min_sim)

    return mat, vocab_idx, idx_vocab

In [224]:
mat, vocab_idx, idx_vocab = word_graph(sents)


n ['기자']
nouns ['기자']
n ['연합뉴스']
nouns ['연합뉴스']
n []
nouns []
n ['오', '패산', '터널', '총격전', '용의자', '검거', '서울', '연합뉴스', '경찰', '관계자', '일', '오후', '서울', '강북구', '오', '패산', '터널', '인근', '사제', '총기', '발사', '경찰', '살해', '용의자', '모씨', '검거', '성', '씨', '검거', '당시', '서바이벌', '게임', '방탄조끼', '헬멧', '착용', '상태']
nouns ['패산', '터널', '총격전', '용의자', '검거', '서울', '연합뉴스', '경찰', '관계자', '오후', '서울', '강북구', '패산', '터널', '인근', '사제', '총기', '발사', '경찰', '살해', '용의자', '모씨', '검거', '검거', '당시', '서바이벌', '게임', '방탄조끼', '헬멧', '착용', '상태']
n ['서울', '연합뉴스', '김은경', '기자', '사제', '총기', '경찰', '살해', '범인', '성모', '씨']
nouns ['서울', '연합뉴스', '김은경', '기자', '사제', '총기', '경찰', '살해', '범인', '성모']
n ['경찰', '성', '씨', '일', '오후', '강북', '경찰서', '인근', '부동산', '업소', '밖', '부동산', '업자', '이모', '씨', '이', '씨', '평소', '말다툼', '자주', '것']
nouns ['경찰', '오후', '강북', '경찰서', '인근', '부동산', '업소', '부동산', '업자', '이모', '평소', '말다툼', '자주']
n ['씨', '시작', '성', '씨', '준비', '사제', '총기', '이', '씨', '발사', '총알', '이', '씨', '총알', '행인', '씨', '배']
nouns ['시작', '준비', '사제', '총기', '발사', '총알', '총알', '행인']
n [

In [225]:
def pagerank(x: np.ndarray, df=0.85, max_iter=50, method="iterative"):

    assert 0 < df < 1

    A = normalize(mat, axis=0, norm="l1")
    N = np.ones(A.shape[0]) / A.shape[0]

    if method == "iterative":
        R = np.ones(A.shape[0])
        for _ in range(max_iter):
            R = df * np.matmul(A, R) + (1 - df) * N
    elif method == "algebraic":
        R = np.linalg.inv((I - df * A))
        R = np.matmul(R, (1 - df) * N)

    return R

In [226]:

def pagerank(x, df=0.85, max_iter=30,method="iterative"):
    assert 0 < df < 1
    if method == "iterative":
        # initialize
        A = normalize(x, axis=0, norm='l1')
        R = np.ones(A.shape[0]).reshape(-1,1)
        bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

        # iteration
        for _ in range(max_iter):
            R = df * (A * R) + bias

    return R

In [227]:
R = pagerank(mat,method ="iterative")

In [228]:
topk = 10
idxs = R.argsort()[-topk:]


In [229]:
keywords = [(idx, R[idx], idx_vocab[idx]) for idx in reversed(idxs)]

TypeError: unhashable type: 'numpy.ndarray'

In [230]:
keywords

[(0.038171474706220364, '서울'),
 (0.03590352211807886, '발사'),
 (0.03567521486237227, '용의자'),
 (0.03458053216612935, '강북구'),
 (0.03455531359160814, '사제'),
 (0.034091948403762985, '의자'),
 (0.034091948403762985, '조사'),
 (0.034091948403762985, '경찰관'),
 (0.03328840487152944, '총기'),
 (0.03296229487665027, '폭행')]